# This notebook has the optimization codes; use fitting_na_16_plotter.ipynb to make plots

In [1]:
import numpy as np
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize, stats
import bluepyopt as bpop
import curve_fitting as cf
import bluepyopt.deapext.algorithms as algo
import generalized_genSim_shorten_time as ggsd
import vclamp_evaluator_HMM as vcl_ev
import pickle
import time
from deap import tools
import multiprocessing
import eval_helper_na12 as eh12

In [5]:
# just adjust these parameters, and then just run every block below to run the optimization

offspring_size = 8
num_generations = 500
output_log_file_name = 'jinan_na12_fitting9.txt'
param_range_file = "./csv_files/param_stats_wide_na12.csv"
mutant_name = "T400RAdult"
mutant_protocol_csv_name = './csv_files/mutant_protocols_CHOP.csv'
initial_baseline_parameters = eh12.get_wt_params_na12()
peak_amp_dict = {"T400RAdult": 0.645, "I1640NAdult": 0.24, "m1770LAdult": 0.4314, "neoWT": 0.748, "T400RAneo": 0.932, "I1640NNeo": 0.28, "m1770LNeo": 1}


In [6]:
import numpy as np
import bluepyopt as bpop
import matplotlib.pyplot as plt

peak_amp_dict = {"T400RAdult": 0.645, "I1640NAdult": 0.24, "m1770LAdult": 0.4314, "neoWT": 0.748, "T400RAneo": 0.932, "I1640NNeo": 0.28, "m1770LNeo": 1}

class Vclamp_evaluator(bpop.evaluators.Evaluator):
    #with peak_amp

    def __init__(self, scaled):
        
        eh12.set_channel()
        
        self.scaled = scaled
        
        # (val, min, max)
        param_range_dict = eh12.read_params_range(param_range_file)
        params_in_name = eh12.get_name_params_str()
        params_not_in_Range_dict = ['qq', 'tq']
        
        eh12.set_param(initial_baseline_parameters)
        
        # diff is mut - wild
        # first get baseline data points:
        gv_slope, v_half, top, bottom = cf.calc_act_obj("na12", is_HMM=False)
        self.act_v_half = v_half
        self.act_slope = gv_slope
        ssi_slope, v_half, top, bottom, _ = cf.calc_inact_obj("na12", is_HMM=False)
        self.inact_v_half = v_half
        self.inact_slope = ssi_slope
        self.tau0 = eh12.find_tau0()
        self.per_cur = eh12.find_persistent_current()
        self.peak_amp = eh12.find_peak_amp()
        
        print("debug: " + str(self.peak_amp))
        
        def init_params():
            param_list = []
            print("here are the name, val, min, max of each parameter")
            for param in params_in_name:
                if param not in params_not_in_Range_dict:
                    print(param)
                    val = param_range_dict[param][0]
                    min_bound = param_range_dict[param][1]
                    max_bound = param_range_dict[param][2]
                    print(val)
                    print((min_bound, max_bound))
                    print("")
                    param_list.append(bpop.parameters.Parameter(param, value=val, bounds=(min_bound, max_bound)))
            return param_list

        print("init called")
        self.objectives = []
        self.objectives.append(bpop.objectives.Objective("V_half_Act"))
        self.objectives.append(bpop.objectives.Objective("V_half_inact"))
        self.objectives.append(bpop.objectives.Objective("slope_Act"))
        self.objectives.append(bpop.objectives.Objective("slope_inact"))
        self.objectives.append(bpop.objectives.Objective("tau0"))
        self.objectives.append(bpop.objectives.Objective("pers_curr"))
        self.objectives.append(bpop.objectives.Objective("peak_amp"))
        self.params = init_params()
        
        goal_dict = eh12.read_mutant_protocols(mutant_protocol_csv_name, mutant_name)
        self.V_half_Act_diff_goal = goal_dict['dv_half_act']
        self.V_half_inact_diff_goal = goal_dict['dv_half_ssi']
        # slopes come in the 100 scale since it's a ratio, so we have to divide by 100
        self.slope_Act_ratio_goal = goal_dict['gv_slope']/100
        self.slope_inact_ratio_goal = goal_dict['ssi_slope']/100
        self.tau0_ratio_goal = goal_dict['tau0']/100
        self.per_cur_ratio_goal = goal_dict['persistent']/100
        self.peak_amp_ratio_goal = peak_amp_dict[mutant_name]
        
        print("\n\n\nhere are the goals:")
        print(self.V_half_Act_diff_goal)
        print(self.V_half_inact_diff_goal)
        print(self.slope_Act_ratio_goal)
        print(self.slope_inact_ratio_goal)
        print(self.tau0_ratio_goal)
        print(self.per_cur_ratio_goal)
        print(self.peak_amp_ratio_goal)
        
    def evaluate_with_lists(self, param_values=[]):
        
        print("evaluate_with_lists is called")
        assert len(param_values) == len(self.params), 'no, they have to be equal...'
        
        currh = ggsd.Activation(channel_name = 'na12').h
        currh.sh_na12 = param_values[0]
        currh.tha_na12 = param_values[1]
        currh.qa_na12 = param_values[2]
        currh.Ra_na12 = param_values[3]
        currh.Rb_na12 = param_values[4]
        currh.thi1_na12 = param_values[5]
        currh.thi2_na12 = param_values[6]
        currh.qd_na12 = param_values[7]
        currh.qg_na12 = param_values[8]
        currh.mmin_na12 = param_values[9]
        currh.hmin_na12 = param_values[10]
        currh.q10_na12 = param_values[11]
        currh.Rg_na12 = param_values[12]
        currh.Rd_na12 = param_values[13]
        currh.thinf_na12 = param_values[14]
        currh.qinf_na12 = param_values[15]
        currh.vhalfs_na12 = param_values[16]
        currh.a0s_na12 = param_values[17]
        currh.zetas_na12 = param_values[18]
        currh.gms_na12 = param_values[19]
        currh.smax_na12 = param_values[20]
        currh.vvh_na12 = param_values[21]
        currh.vvs_na12 = param_values[22]
        currh.Ena_na12 = param_values[23]
        currh.Ena_na12 = 55
        
        
        try:
            gv_slope, act_v_half, act_top, act_bottom = cf.calc_act_obj("na12", is_HMM=False)
            ssi_slope, inact_v_half, inact_top, inact_bottom, tau999 = cf.calc_inact_obj("na12", is_HMM=False)
            tau0 = eh12.find_tau0()
            per_cur = eh12.find_persistent_current()
        except:
            return [9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999]

        V_half_Act_diff = act_v_half - self.act_v_half
        V_half_inact_diff = inact_v_half - self.inact_v_half
        gv_slope_ratio = gv_slope/self.act_slope
        ssi_slope_ratio = ssi_slope/self.inact_slope
        tau0_ratio = tau0/self.tau0
        per_cur_ratio = per_cur/self.per_cur
        
        try:
            # eliminate outliers
            act = ggsd.Activation(channel_name = 'na12')
            act.genActivation()
            norm_act_y_val = sorted(list(act.gnorm_vec))
            act_fitted = eh12.get_fitted_act_conductance_arr(act.v_vec, gv_slope, act_v_half, act_top, act_bottom)

            inact = ggsd.Inactivation(channel_name = 'na12')
            inact.genInactivation()
            norm_inact_y_val = sorted(list(inact.inorm_vec))
            inac_fitted = eh12.get_fitted_inact_current_arr(inact.v_vec, ssi_slope, inact_v_half, inact_top, inact_bottom)
        except:
            return [9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999]
        
        try: 
            mutant_peak_amp = eh12.find_peak_amp()
            peak_amp_ratio = mutant_peak_amp/self.peak_amp

            print("debug: " + str(peak_amp_ratio))
        except:
            return [9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999]

        
        if self.scaled:            
            return [(V_half_Act_diff/self.V_half_Act_diff_goal - 1)**2 * 1000,
                   (V_half_inact_diff/self.V_half_inact_diff_goal - 1)**2 * 1000,
                   (gv_slope_ratio/self.slope_Act_ratio_goal - 1)**2 * 1000,
                   (ssi_slope_ratio/self.slope_inact_ratio_goal - 1)**2 * 1000,
                   (tau0_ratio/self.tau0_ratio_goal - 1)**2 * 1000,
                   (per_cur_ratio/self.per_cur_ratio_goal - 1)**2 * 1000,
                   (peak_amp_ratio/self.peak_amp_ratio_goal - 1)**2 * 1000]
        else:
            return [(V_half_Act_diff - self.V_half_Act_diff_goal)**2,
                   (V_half_inact_diff - self.V_half_inact_diff_goal)**2,
                   (gv_slope_ratio - self.slope_Act_ratio_goal)**2,
                   (ssi_slope_ratio - self.slope_inact_ratio_goal)**2,
                   (tau0_ratio - self.tau0_ratio_goal)**2,
                   (per_cur_ratio - self.per_cur_ratio_goal)**2,
                   (peak_amp_ratio - self.peak_amp_ratio_goal)**2]
        

In [7]:
# if scaled, then we will use the scaled scoring method that assigns equal weights to each parameter. Otherwise,
#      we will use natural weights
evaluator = Vclamp_evaluator(scaled = False)

debug: -0.3006560575214934
init called
here are the name, val, min, max of each parameter
sh
8.0
(0.1, 50.0)

tha
-28.76
(-150.0, -1.0)

qa
5.41
(0.1, 100.0)

Ra
0.3282
(0.005, 2.0)

Rb
0.1
(0.005, 2.0)

thi1
-37.651
(-150.0, -1.0)

thi2
-30.0
(-150.0, -1.0)

qd
0.5
(0.0, 20.0)

qg
1.5
(0.0, 20.0)

mmin
0.02
(5e-05, 0.5)

hmin
0.01
(5e-05, 0.5)

q10
2.0
(0.0, 20.0)

Rg
0.01
(0.0005, 0.2)

Rd
0.02657
(0.0005, 0.2)

thinf
-45.0
(-150.0, -1.0)

qinf
7.69
(0.5, 100.0)

vhalfs
-60.0
(-150.0, -1.0)

a0s
0.0003
(1.5e-06, 0.1)

zetas
12.0
(0.1, 80.0)

gms
0.2
(0.003, 3.0)

smax
10.0
(0.1, 150.0)

vvh
-58.0
(-150.0, -1.0)

vvs
2.0
(-50.0, 60.0)

Ena
55.0
(45.0, 65.0)




here are the goals:
-1.9
-3.6
1.04
0.98
1.524
1.0
0.645


In [8]:
cur_log_file = output_log_file_name

gen_counter = 0
best_indvs = []
cp_freq = 1
old_update = algo._update_history_and_hof
def my_update(halloffame, history, population):
    global gen_counter,cp_freq
    if halloffame is not None:
        halloffame.update(population)
    
    if halloffame:
        best_indvs.append(halloffame[0])
        print(halloffame[0])
        f = open(cur_log_file, 'a')
        f.write(str(halloffame[0]) + '\n')
        f.close()
        #eh12.make_act_plots(halloffame[0])
        #eh12.make_inact_plots(halloffame[0])
    gen_counter = gen_counter+1
    print("Current generation: ", gen_counter)
    if gen_counter%cp_freq == 0:
        fn = '.pkl'
        save_logs(fn,best_indvs,population)

def my_record_stats(stats, logbook, gen, population, invalid_count):
    '''Update the statistics with the new population'''
    record = stats.compile(population) if stats is not None else {}
    logbook.record(gen=gen, nevals=invalid_count, **record)
    f = open(cur_log_file, 'a')
    f.write(str(logbook) + '\n\n\n')
    f.close()
    print('log: \n', logbook, '\n')
    output = open("log.pkl", 'wb')
    pickle.dump(logbook, output)
    output.close()

def save_logs(fn, best_indvs, hof):
    output = open("indv"+fn, 'wb')
    pickle.dump(best_indvs, output)
    output.close()
    output = open("hof"+fn, 'wb')
    pickle.dump(hof, output)


In [9]:
#hof = tools.HallOfFame(1, similar=np.array_equal)
hof = tools.ParetoFront()
algo._update_history_and_hof = my_update
algo._record_stats = my_record_stats
pool = multiprocessing.Pool(processes=64)
deap_opt = bpop.optimisations.DEAPOptimisation(evaluator, offspring_size=offspring_size, hof = hof, map_function=pool.map)
#, map_function=pool.map
#deap_opt = bpop.optimisations.DEAPOptimisation(evaluator, offspring_size=5, hof = hof)
cp_file = './cp.pkl'

evaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is called
evaluate_with_lists is calledevaluate_with_lists is called

evaluate_with_lists is called
evaluate_with_lists is called





/global/u1/j/jinan/dec18/SimulatingVariants/CHOP/files_for_optimization/generalized_genSim_shorten_time.py:210: RuntimeWarning: overflow encountered in exp
  return Gmax * (vm - self.vrev) / (1 + np.exp((v_half - vm) / s))
/global/homes/j/jinan/.local/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/global/u1/j/jinan/dec18/SimulatingVariants/CHOP/files_for_optimization/generalized_genSim_shorten_time.py:218: RuntimeWarning: overflow encountered in exp
  norm_g.append(1 / (1 + np.exp(-(volt - self.v_half) / self.s)))
/global/u1/j/jinan/dec18/SimulatingVariants/CHOP/files_for_optimization/curve_fitting.py:17: RuntimeWarning: overflow encountered in exp
  return bottom +  ((top - bottom) / (1.0 + np.exp((v_half - x)/slope)))
/global/homes/j/jinan/.local/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  

debug: 0.06650211224096136
debug: 1.0201943823944981
debug: 0.08320194453020563
debug: 0.30287784726662675
debug: 1.4637029891238873
debug: 0.279236332444858
debug: 0.01131496311724835
debug: 0.24985137794278778
evaluate_with_lists is called
evaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is called


evaluate_with_lists is called
evaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is called




/global/homes/j/jinan/.local/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/global/homes/j/jinan/.local/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)
/global/homes/j/jinan/.local/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/global/homes/j/jinan/.local/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/global/homes/j/jinan/.local/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)
/global/homes/j/jinan/.local/li

debug: 0.12073014955931087
debug: 0.1944011504180997
debug: 0.6139331045298007
debug: 1.3316770726620406
debug: 0.05010995626067765
debug: 1.7808584302483448
debug: 0.03682275885078055
debug: 0.282285297629384
evaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is called



evaluate_with_lists is called





...

debug: 0.8627515626671105
debug: 3.63506157456106
debug: 0.7858522312341064
debug: 0.929173374947719
debug: 0.9387250219227429
debug: 1.0396331343256373
debug: 0.8453728462365496
debug: 1.4254425073351689
evaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is called


evaluate_with_lists is called






/global/u1/j/jinan/dec18/SimulatingVariants/CHOP/files_for_optimization/curve_fitting.py:17: RuntimeWarning: overflow encountered in exp
  return bottom +  ((top - bottom) / (1.0 + np.exp((v_half - x)/slope)))


debug: 0.9357351013074028
debug: 2.4531568406362627
debug: 1.4326189735213752
debug: 0.3873084292601717
debug: 0.7907139392039823
debug: 1.826344592292922
debug: 0.6834185020220434
debug: 2.061262627983844
evaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is called

evaluate_with_lists is calledevaluate_with_lists is called
evaluate_with_lists is called
evaluate_with_lists is called



debug: 1.2369529417693936
debug: 0.24518804103600741
debug: 0.9346469538335392
debug: 0.32927245277133527
debug: 0.2944221802339022
debug: 2.159120602649462
debug: 1.8344529969432357
debug: 1.1780294310477146
evaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is called
evaluate_with_lists is called








/global/u1/j/jinan/dec18/SimulatingVariants/CHOP/files_for_optimization/generalized_genSim_shorten_time.py:210: RuntimeWarning: overflow encountered in exp
  return Gmax * (vm - self.vrev) / (1 + np.exp((v_half - vm) / s))
/global/u1/j/jinan/dec18/SimulatingVariants/CHOP/files_for_optimization/generalized_genSim_shorten_time.py:218: RuntimeWarning: overflow encountered in exp
  norm_g.append(1 / (1 + np.exp(-(volt - self.v_half) / self.s)))


debug: 1.0324630371802348
debug: 1.924975231232662
debug: 0.13876334421746392
debug: 0.15355146512463008
debug: 0.13534045614071075
debug: 1.1855874969950135
debug: 0.0006399800679589753
debug: 0.4756788413554545
evaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is calledevaluate_with_lists is called


In [ ]:
start_time = time.time()
#pop, hof, log, hst = deap_opt.run(max_ngen=5, cp_filename=cp_file)
pop, hof, log, hst = deap_opt.run(max_ngen=num_generations, cp_filename=None)
end_time = time.time()
print(end_time - start_time)

[12.839172600356566, -24.580019991159688, 67.34404119132684, 0.1710521049187763, 0.0382978070805414, -147.83056373620298, -37.417570487422466, 4.991184513068456, 2.1897725458871875, 0.3124198019720305, 0.17224421090504263, 1.3903075706169465, 0.032345292176422576, 0.10571238961007856, -124.94640301285818, 27.65498646345867, -43.97310084939434, 0.04547098095212132, 25.827941134347345, 1.4228917294683257, 3.6428231870348613, -92.40299139054099, -3.6989452870016493, 48.76078609502626]
Current generation:  1
log: 
 gen	nevals	avg    	std    	min   	max    
1  	8     	20448.3	23040.7	1180.2	76349.9 

[6.8047757812088205, -73.66250712787368, 76.40108443576374, 0.513861494971202, 0.9933929987484222, -76.83560992730538, -55.39351840963323, 15.756467154564511, 1.0712835689884037, 0.029325286284381205, 0.4178907637047389, 8.655341358101067, 0.15736735026060628, 0.10018601354399864, -83.63730808583459, 21.494335366923742, -109.83131455766849, 0.0945271516493489, 73.31233649012555, 0.0946781791515